# Homework 1 , Bahadır Erdem , 21070001048


# Standard Normalization by Vectorization


For broadcasting: https://www.youtube.com/watch?v=tuKHsfAehz4
* Broadcasting enables operations to be performed element-wise on arrays of different shapes without the need for the arrays to have the same shape. 
* Broadcasting is useful when we want to perform operations between arrays of different shapes without explicitly reshaping or duplicating the data.

In [1]:
#Importing the libraries that I use in this project
import numpy as np
import matplotlib.pyplot as plt

In [3]:

def stdnorm(X):
    return (X - np.mean(X, axis=0, keepdims=True)) / np.std(X, axis=0, keepdims=True)
N = 10
D = 3
x_org = np.arange(1, N+1).reshape(N,1) @ np.arange(1, D+1).reshape(1,D)
x_org = np.arange(1, N+1)[:, np.newaxis] * np.arange(1, D+1)   #I am not sure about this line. I will look it again later.
print(x_org.shape)
print(x_org)
x_norm = stdnorm(x_org)
print(x_norm)
print('Mean:', np.mean(x_norm, axis=0))
print('Std:', np.std(x_norm, axis=0))


(10, 3)
[[ 1  2  3]
 [ 2  4  6]
 [ 3  6  9]
 [ 4  8 12]
 [ 5 10 15]
 [ 6 12 18]
 [ 7 14 21]
 [ 8 16 24]
 [ 9 18 27]
 [10 20 30]]
[[-1.5666989  -1.5666989  -1.5666989 ]
 [-1.21854359 -1.21854359 -1.21854359]
 [-0.87038828 -0.87038828 -0.87038828]
 [-0.52223297 -0.52223297 -0.52223297]
 [-0.17407766 -0.17407766 -0.17407766]
 [ 0.17407766  0.17407766  0.17407766]
 [ 0.52223297  0.52223297  0.52223297]
 [ 0.87038828  0.87038828  0.87038828]
 [ 1.21854359  1.21854359  1.21854359]
 [ 1.5666989   1.5666989   1.5666989 ]]
Mean: [-1.11022302e-16 -1.11022302e-16 -1.11022302e-16]
Std: [1. 1. 1.]


# The XOR Problem
Design and train a neural network with the minimum number of
neurons that predicts XOR outputs with 100% accuracy. If you use more neurons than the minimum
number you get 5 points less. You must clearly show that your model works. Create the XOR dataset
yourself. You are free to apply any learning method (shortly explain what you apply), but you MUST
use only numpy for the computations. No other package is allowed for the computations, you can use
other packages only for visualization or reporting. You are free to use any code from the lab sessions.

This website helped a lot: https://www.sharpsightlabs.com/blog/python-perceptron-from-scratch/

In [1]:
# The XOR Dataset
import numpy as np
import matplotlib.pyplot as plt

# XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])

class TwoLayerMLP:
    def __init__(self, input_size, hidden_size, output_size, reg_lambda):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.reg_lambda = reg_lambda
        self.init_weights()
        self.zero_grad()

    def init_weights(self):
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * 0.01
        self.b2 = np.zeros((1, self.output_size))

    def zero_grad(self):
        self.dW1 = np.zeros_like(self.W1)
        self.db1 = np.zeros_like(self.b1)
        self.dW2 = np.zeros_like(self.W2)
        self.db2 = np.zeros_like(self.b2)

    def update_weights(self, learning_rate):
        self.W1 -= learning_rate * self.dW1
        self.b1 -= learning_rate * self.db1
        self.W2 -= learning_rate * self.dW2
        self.b2 -= learning_rate * self.db2

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def sigmoid_derivative(self, Z):
        s = self.sigmoid(Z)
        return s * (1 - s)

    def forward(self, X, Y=None):
        self.X = X
        if Y is not None:
            self.Y = Y

        self.Z1 = np.dot(X, self.W1) + self.b1
        self.H1 = self.sigmoid(self.Z1)
        self.Z2 = np.dot(self.H1, self.W2) + self.b2
        self.H2 = self.Z2
        return self.H2

    def compute_loss(self, Y_pred=None, Y=None):
        if Y_pred is None:
            Y_pred = self.H2
        if Y is None:
            Y = self.Y

        batch_size = Y_pred.shape[0]
        data_loss = np.sum((Y_pred - Y) ** 2) / batch_size
        reg_loss = (self.reg_lambda / 2) * (np.sum(np.square(self.W1)) + np.sum(np.square(self.W2))) / batch_size
        loss = data_loss + reg_loss

        return loss

    def backward(self):
        Y_pred = self.H2
        Y = self.Y
        X = self.X

        dZ2 = (Y_pred - Y) * self.sigmoid_derivative(self.Z2)
        self.dW2 = (np.dot(self.H1.T, dZ2) + self.reg_lambda * self.W2)
        self.db2 = np.sum(dZ2, axis=0, keepdims=True)

        dZ1 = np.dot(dZ2, self.W2) * self.sigmoid_derivative(self.Z1)
        self.dW1 = (np.dot(X.T, dZ1) + self.reg_lambda * self.W1)
        self.db1 = np.sum(dZ1, axis=0, keepdims=True)

    def train(self, X, y, learning_rate=0.01, epochs=1000):
        losses = []
        for epoch in range(epochs):
            # Forward pass
            self.forward(X, y)
            # Compute loss
            loss = self.compute_loss()
            losses.append(loss)
            # Backward pass
            self.backward()
            # Update weights
            self.update_weights(learning_rate)
        return losses

    def predict(self, X):
        return np.where(self.forward(X) >= 0.5, 1, 0)

# Instantiate the model
model = TwoLayerMLP(input_size=2, hidden_size=2, output_size=1, reg_lambda=0.01)

# Train the model
losses = model.train(X, y, learning_rate=0.1, epochs=100)

# Plot the loss curve
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.show()

# Evaluate accuracy
predictions = model.predict(X)
accuracy = np.mean(predictions == y)
print("Accuracy:", accuracy)

# Plot the accuracy with the epochs
plt.plot(np.arange(100), losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.show()


ValueError: shapes (4,4) and (2,1) not aligned: 4 (dim 1) != 2 (dim 0)